Dask Approach to Non-SQL PTM Queries
--

v01: move away from classes, try to organize the computation 
primarily in dask.

In [1]:
import postproc_dask as post
import six

In [2]:
try:
    client.close()
except NameError:
    pass

In [3]:
import multiprocessing.popen_spawn_posix #  https://github.com/dask/distributed/issues/4168
import dask
import dask.dataframe as dd
import dask.bag as db

from dask.distributed import Client
client=Client(n_workers=24,threads_per_worker=1)
client.cluster

In [4]:
import matplotlib.pyplot as plt
import conc_figure
import six
import stompy.plot.cmap as scmap
from stompy.spatial import proj_utils
from matplotlib import cm
cmap=cm.CMRmap_r
cmap=scmap.cmap_clip(cmap,0.03,1.0)

%matplotlib notebook

In [5]:
import os
import glob
import numpy as np
import pandas as pd
import xarray as xr
import re

from stompy.grid import unstructured_grid
from stompy import utils, memoize
from stompy.model.fish_ptm import ptm_config, ptm_tools
from stompy.model.suntans import sun_driver
from scipy.stats import spearmanr

import stompy.plot.cmap as scmap
from scipy import stats
import seaborn as sns
turbo=scmap.load_gradient('turbo.cpt')

Overall Process
===

1. SUNTANS hydro runs
2. SUNTANS average output
3. ptm-formatted average output
4. PTM runs
5. Load data

The top-level query is something like *generate a map of concentrations for...*

filter on:
 - sources $x$
 - settling classes $y$
 - vertical positions $z$
 - horizontal positions $h$

weighted by

 - loading data 
 - age
 
mapped by one of ...

 - bounding box
 - put on hydro grid
 - put on regular grid

and possibly smoothed.

In [6]:
# Experiment level configuration -- small, all python native data.
import cfg_v01 
cfg=dict(cfg_v01.cfg)

In [7]:
# Load the grid into... grid
hydro_path=cfg['sun_paths'][0]
ptm_ds=xr.open_dataset(os.path.join(hydro_path,"ptm_average.nc_0000.nc"))
grid=unstructured_grid.UnstructuredGrid.read_ugrid(ptm_ds,dialect='fishptm')
ptm_ds.close()   

# distribute to workers ahead of time.
grid_d=client.scatter(grid)
cfg['grid_d']=grid_d # too far?

In [8]:
# So far this is only used locally.  Slow to compute (15s)
Msmooth=grid.smooth_matrix()
Msmooth_K=grid.smooth_matrix(K=100*np.ones(grid.Nedges()))

INFO:utils:62340/99089
INFO:utils:61226/99089


In [9]:
# Or could make this delayed and have it execute on each client?
load_data_d=client.scatter(post.get_load_data())
cfg['load_data_d']=load_data_d

In [10]:
# Hydro timestamps
cfg['hydro_timestamps']=post.load_hydro_timestamps(cfg['sun_paths'])

In [11]:
bc_ds_d=client.scatter(post.bc_ds(cfg=cfg))
cfg['bc_ds_d']=bc_ds_d

In [12]:
areas=grid.cells_area()    

In [13]:
# Set this a bit early, as it controls cache location which 
# is needed for some testing.
cfg['manta_out_dir']="manta_sets_20210726d"

# now the 021b runs.
# c suffix: use tighter -0.095 z filter.
# 20200227: start of v03
# 20200302: start of v04
# 20200305: try larger window -- extra 24 hours in both directions.
# 20200312: new sources in ptm data, with v02 stormwater concs.
# 20200330: more new sources.
# 20210210a: dask code, new hydro, new ptm. new year.
# 20210317a: new ptm runs with no SJ loss...
#            has the w_s bug
# 20210317b: fixed w_s bug
# 20210317c: switch to weathered loads
# 20210708c: including beaching and deposition counts.
os.makedirs(cfg['manta_out_dir'],exist_ok=True)

Beaching and Deposition
---

Beaching and deposition seem to be more likely candidates for scale offsets 
than sinking.

In [14]:
# Testing query_group_particles:
six.moves.reload_module(post)
if 0:
    run=cfg['ptm_run_paths'][0]
    grps=post.run_to_group_paths(run)
    grp=grps[0]
    load_data=post.get_load_data()
    bc_ds=post.bc_ds(cfg=cfg)

    df=post.query_group_particles(grp,
                               dict(t_min=np.datetime64('2017-06-30'),
                                    t_max=np.datetime64('2017-07-04'),
                                    age_max=np.timedelta64(60,'D')),
                               load_data=load_data,
                               bc_ds=bc_ds)

    df2=post.query_group_particles(None,
                                   dict(t_min=np.datetime64('2017-06-30'),
                                        t_max=np.datetime64('2017-07-04'),
                                        age_max=np.timedelta64(60,'D')),
                                   load_data=load_data,
                                   bc_ds=bc_ds)


In [30]:
#%time df=post.particles_for_date("2017-10-21 00:00:00",cfg=cfg,cache=False)

len(df) # 41522151, took 39 minutes with 8 processes, warm-ish.
# 22 minutes warm, on 24 processes.
# Compare that to 17.4 minutes via mp.
# After updating the dask code to follow the simpler sequence now that
# info is precomputed...

None
Will not repartition
CPU times: user 3min 40s, sys: 51.9 s, total: 4min 32s
Wall time: 21min 1s


Next steps
--


1. Recreate some of the figures from before, including on-grid smoothing.  
  a. Sample plot: from the powerpoint. 2017-08-30 to 2017-09-14. Surface particles
     max age of 10 days.   *This plot is similarish -- not exactly the same but close
     enough to rule out fundamental errors*
2. Pull out manta samples as before. Maybe skip putting it on the grid, just
   query a radius.
  a. Implement in this notebook
  b. Move all of this to a module.

In [15]:
ll2utm=proj_utils.mapper('WGS84','EPSG:26910')
# Load the manta data, clean it up
manta_fn='manta_summary-v03.csv'
manta=pd.read_csv(manta_fn)
# manta.head()

In [16]:
godwin_offset_h=post.godwin_offset_h
godwin=post.godwin

In [33]:
%load_ext line_profiler

In [17]:
def predict_mantas_multi(manta,taus,
                         Msmooths=[Msmooth],
                         steps=[30],
                         storm_factors=[1.0],
                         shore_factors=[1e-2],
                         bed_factors=[1e-2],
                         cfg=cfg):
    # manta: pd.DataFrame 
    # tau: np.timedelta64 giving e-folding time scale
    # of loss.
    # storm_factor: scale stormwater concentrations by this
    #  factor.
    # shore_factors: particle weight derated by (1-shore_factor)**hits
    # bed_factors: particle weighte derated by (1-bed_factor)**hits
    preds=[]
    
    # Force mantas to be sorted, so that data from the same day
    # is grouped together.
    manta=manta.sort_values('DATE')
    last_DATE=None
    df=None

    for idx,rec in utils.progress(manta.iterrows()):
        print(idx)
        # since we're just pulling out whole days, cache by day.
        # and pull through the end of that day
        # 5s, all in read_parquet
        if rec.DATE==last_DATE:
            pass
        else:
            df=post.particles_for_date(rec.DATE,cfg=cfg)
            last_DATE=rec.DATE

        # Trawl-specific processing:
        # Pull subset of cells
        # add a cell weighting, and a time weighting for the tidal filter
        for Msmooth_i,Msmooth in enumerate(Msmooths):
            print(f"  smooth: {Msmooth_i}")
            for step in steps:
                print(f"    steps: {step}")
                cells,weights=post.rec_to_cell_weights(rec,areas=areas,grid=grid,
                                                       Msmooth=Msmooth,smooth=step,thresh=1e-5)
                df_spatial=pd.DataFrame(dict(cell=cells,weight_spatial=weights)).set_index('cell')
                # 1.3s. selecting a subset of rows, and adding a column with spatial weight.
                # Using a dictionary might get this down to 60% of original
                # going through numpy  might get down to 40% of original.
                if 0:
                    tdf=df.join(df_spatial,on='cell',how='inner')
                else:
                    # trying numpy approach
                    df_spatial_sort=df_spatial.sort_index()
                    space_cell=df_spatial_sort.index.values
                    space_weight=df_spatial_sort['weight_spatial'].values
                    df_cell=df.cell.values
                    df_idx=np.searchsorted(space_cell,df_cell).clip(0,len(space_cell)-1)
                    sel=space_cell[df_idx]==df_cell
                    tdf=df[sel].copy()
                    tdf['weight_spatial']=space_weight[df_idx[sel]]                    

                t_sample=np.datetime64(rec['DATE'])    
                t_center = t_sample+np.timedelta64(8,'h') + np.timedelta64(12,'h')
                delta_hours=((tdf['time']-t_center)/np.timedelta64(1,'h')).astype(np.int32)
                weight_time=godwin[delta_hours+godwin_offset_h]
                age=tdf['time'].values - tdf['rel_time'].values
    
                for storm_factor in storm_factors:
                    print(f"      storm_factor: {storm_factor}")
                    group_weight=post.group_weights(tdf,storm_factor)

                    for tau in taus:
                        tau_s=tau/np.timedelta64(86400,'s')
                        print(f"        tau={tau_s} days")

                        if tau_s>0:
                            decay=np.exp( -age/tau )
                        else:
                            decay=1.0
                            
                        for bed_factor in bed_factors:
                            print(f"          bed={bed_factor}")

                            bed_weight=(1-bed_factor)**tdf['bed_hits']
                            
                            for shore_factor in shore_factors:
                                print(f"            shore={shore_factor}")
                                shore_weight=(1-shore_factor)**tdf['shore_hits']
                                pred=dict(idx=idx,storm_factor=storm_factor,tau=tau,step=step,
                                          Msmooth_i=Msmooth_i,shore_factor=shore_factor,
                                          bed_factor=bed_factor)
                                pred['idx']=idx
                                
                                weights=tdf['weight_spatial']*weight_time*decay*group_weight*bed_weight*shore_weight

                                # tdf['mp_per_particle'] # the number of physical particles represented by the virtual particle
                                #   This is the contribution from this one
                                #   virtual particle the cell concentration
                                C=( weights * tdf['mp_per_particle'] / areas[tdf['cell']] ).sum()
                                pred['C_model']=C
                                preds.append(pred)

    df_pred=pd.DataFrame(preds)
    manta_pred=df_pred.join(manta,on='idx')
    return manta_pred

In [18]:
def metrics(grp):
    obs=grp['part_per_m2_nofiber'].values
    mod=grp['C_model'].values
    
    skill={}
    skill['rho'],skill['rho_p']=spearmanr(obs, mod)
    skill['norm_std']=mod.std() / obs.std()
    skill['norm_mag']=mod.mean() / obs.mean()
    skill['rel_std_obs']=obs.std() / obs.mean()
    skill['rel_std_mod']=mod.std() / mod.mean()
    
    # log-transformed
    min_nonzero=obs[obs>0].min()
    r_log=np.corrcoef(np.log(obs.clip(min_nonzero)),
                      np.log(mod.clip(min_nonzero)))[0,1]
    skill['r_log']=r_log
    
    # Same but drop top/bottom 2 outliers:
    obs_rank=np.argsort(np.argsort(obs))
    sel=(obs_rank>2)&(obs_rank<len(obs)-2)
    r=np.corrcoef(obs[sel], mod[sel])[0,1]
    skill['r_sub']=r
    
    skill['norm_std_sub']=mod[sel].std() / obs[sel].std()
    skill['norm_mag_sub']=mod[sel].mean() / obs[sel].mean()
    skill['rel_std_obs_sub']=obs[sel].std() / obs[sel].mean()
    skill['rel_std_mod_sub']=mod[sel].std() / mod[sel].mean()

    return pd.Series(skill)
   

In [ ]:
if 0:
    # Evaluate effect of smoothing on correlation metrics
    # This one testing evaluation of multiple smooths, steps,
    # tau, etc. in one call.
    taus=[np.timedelta64(int(25*86400),'s')]
    Msmooths=[Msmooth,Msmooth_K]
    mtypes=['grid','constant']
    steps=[0,-10,-100,-1000,-10000,-100000]

    manta_pred=predict_mantas_multi(manta,taus=taus,
                                    Msmooths=Msmooths,
                                    steps=steps,
                                    storm_factors=[1.0])
    manta_pred['Msmooth_label']=[ mtypes[i] for i in manta_pred['Msmooth_i'].values]

    manta_pred.groupby(['Msmooth_label','step','tau','storm_factor']).apply(metrics)

    taus=[np.timedelta64(int(d*86400),'s') for d in [1,2,5,10,20,30,45,60]]
    Msmooths=[Msmooth]
    mtypes=['grid']
    steps=[-10000]

    manta_pred_tau=predict_mantas_multi(manta,taus=taus,
                                        Msmooths=Msmooths,
                                        steps=steps,
                                        storm_factors=[1])
    manta_pred_tau['Msmooth_label']=[ mtypes[i] for i in manta_pred_tau['Msmooth_i'].values]

    manta_pred_tau.groupby(['Msmooth_label','step','tau','storm_factor']).apply(metrics)

In [22]:
# Master set of predictions
taus=[np.timedelta64(int(d*86400),'s') for d in [5,10,20,30,60]]
Msmooths=[Msmooth] # ,Msmooth_K]
mtypes=['grid'] # ,'constant']
steps=[-10,-1000,-100000]
storm_factors=[1.0]
# First go round suggested that 0.001 was too small an increment.
shore_factors=[0,0.01,0.05,0.1]
bed_factors=[0.,0.01,0.05,0.1]

In [23]:
manta_pred_master=predict_mantas_multi(manta,taus=taus,
                                       Msmooths=Msmooths,
                                       steps=steps,
                                       storm_factors=storm_factors,
                                       bed_factors=bed_factors,
                                       shore_factors=shore_factors)
manta_pred_master['Msmooth_label']=[ mtypes[i] for i in manta_pred_master['Msmooth_i'].values]

manta_pred_master.groupby(['Msmooth_label','step','tau','storm_factor']).apply(metrics)
manta_pred_master['tau']=manta_pred_master['tau']/np.timedelta64(86400,'s')
manta_pred_master.to_parquet('manta_pred_master_v03.parquet')

0
  smooth: 0
    steps: -10
      storm_factor: 1.0
        tau=5.0 days
          bed=0.0
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.01
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.05
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.1
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
        tau=10.0 days
          bed=0.0
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.01
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.05
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.1
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
        tau=20.0 days
          bed=0.0
            

INFO:utils:1


1
  smooth: 0
    steps: -10
      storm_factor: 1.0
        tau=5.0 days
          bed=0.0
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.01
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.05
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.1
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
        tau=10.0 days
          bed=0.0
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.01
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.05
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.1
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
        tau=20.0 days
          bed=0.0
            

INFO:utils:2


2
  smooth: 0
    steps: -10
      storm_factor: 1.0
        tau=5.0 days
          bed=0.0
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.01
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.05
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.1
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
        tau=10.0 days
          bed=0.0
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.01
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.05
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.1
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
        tau=20.0 days
          bed=0.0
            

...

63
  smooth: 0
    steps: -10
      storm_factor: 1.0
        tau=5.0 days
          bed=0.0
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.01
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.05
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.1
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
        tau=10.0 days
          bed=0.0
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.01
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.05
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.1
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
        tau=20.0 days
          bed=0.0
           

INFO:utils:63


62
  smooth: 0
    steps: -10
      storm_factor: 1.0
        tau=5.0 days
          bed=0.0
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.01
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.05
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.1
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
        tau=10.0 days
          bed=0.0
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.01
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.05
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.1
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
        tau=20.0 days
          bed=0.0
           

INFO:utils:64


64
  smooth: 0
    steps: -10
      storm_factor: 1.0
        tau=5.0 days
          bed=0.0
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.01
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.05
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.1
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
        tau=10.0 days
          bed=0.0
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.01
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.05
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.1
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
        tau=20.0 days
          bed=0.0
           

In [34]:
# Profiling:
# 386s...
%lprun -f predict_mantas_multi predict_mantas_multi(manta.iloc[:1,:],taus=taus,Msmooths=Msmooths,steps=steps,storm_factors=storm_factors,bed_factors=bed_factors,shore_factors=shore_factors)

0
  smooth: 0
    steps: -10
      storm_factor: 1.0
        tau=5.0 days
          bed=0.0
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.01
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.05
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.1
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
        tau=10.0 days
          bed=0.0
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.01
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.05
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
          bed=0.1
            shore=0
            shore=0.01
            shore=0.05
            shore=0.1
        tau=20.0 days
          bed=0.0
            

In [24]:
# manta_pred_master=pd.read_parquet('manta_pred_master_v03.parquet')

In [25]:
all_metrics=manta_pred_master.groupby(['Msmooth_label','step','tau','storm_factor','bed_factor','shore_factor']).apply(metrics)

In [30]:
# Best overall rho:
#  tau=5, storm=1, grid smooth, step=-100000 => rho=0.74
#  insensitive to tau. mag still something like 14x too large for tau=60,
#  interestingly, even with tau=5, magnitude still 7x too large.
# Best overall r_log:
#  shore_factor is pinned to 0.1, bed_factor 0.00, tau all over the place.
all_metrics.sort_values('r_log',ascending=False).head(10)

rho  \
Msmooth_label step    tau  storm_factor bed_factor shore_factor             
grid          -100000 10.0 1.0          0.00       0.10          0.738024   
                      20.0 1.0          0.00       0.10          0.740166   
                      30.0 1.0          0.00       0.10          0.744362   
                      60.0 1.0          0.00       0.10          0.741652   
                      5.0  1.0          0.00       0.10          0.744231   
                      10.0 1.0          0.01       0.10          0.737019   
                      5.0  1.0          0.01       0.10          0.743750   
                      20.0 1.0          0.01       0.10          0.741608   
                      5.0  1.0          0.00       0.05          0.740603   
                                        0.05       0.10          0.738549   

                                                                        rho_p  \
Msmooth_label step    tau  storm_factor bed_factor shore_factor                 
grid          -100000 10.0 1.0          0.00       0.10          2.312209e-12   
                      20.0 1.0          0.00       0.10          1.850924e-12   
                      30.0 1.0          0.00       0.10          1.189313e-12   
                      60.0 1.0          0.00       0.10          1.584081e-12   
                      5.0  1.0          0.00       0.10          1.206022e-12   
                      10.0 1.0          0.01       0.10          2.564873e-12   
                      5.0  1.0          0.01       0.10          1.269230e-12   
                      20.0 1.0          0.01       0.10          1.591375e-12   
                      5.0  1.0          0.00       0.05          1.768278e-12   
                                        0.05       0.10          2.190016e-12   

                                                                 norm_std  \
Msmooth_label step    tau  storm_factor bed_factor shore_factor             
grid          -100000 10.0 1.0          0.00       0.10          0.466292   
                      20.0 1.0          0.00       0.10          0.512613   
                      30.0 1.0          0.00       0.10          0.530965   
                      60.0 1.0          0.00       0.10          0.551753   
                      5.0  1.0          0.00       0.10          0.396557   
                      10.0 1.0          0.01       0.10          0.457553   
                      5.0  1.0          0.01       0.10          0.390353   
                      20.0 1.0          0.01       0.10          0.501549   
                      5.0  1.0          0.00       0.05          0.854309   
                                        0.05       0.10          0.371893   

                                                                 norm_mag  \
Msmooth_label step    tau  storm_factor bed_factor shore_factor             
grid          -100000 10.0 1.0          0.00       0.10          1.350533   
                      20.0 1.0          0.00       0.10          1.511056   
                      30.0 1.0          0.00       0.10          1.581375   
                      60.0 1.0          0.00       0.10          1.666921   
                      5.0  1.0          0.00       0.10          1.136533   
                      10.0 1.0          0.01       0.10          1.309109   
                      5.0  1.0          0.01       0.10          1.109496   
                      20.0 1.0          0.01       0.10          1.453519   
                      5.0  1.0          0.00       0.05          1.970864   
                                        0.05       0.10          1.038223   

                                                                 rel_std_obs  \
Msmooth_label step    tau  storm_factor bed_factor shore_factor                
grid          -100000 10.0 1.0          0.00       0.10             4.739295   
                      20.0 1.0          0.00       0.10             4.739295   
  

In [28]:
# Bed factor from 0 to 0.1 gives norm_mag of 10.4 down to 7.
all_metrics.loc[('grid',-1000,60,1.0,slice(None),0.0),:]

rho  \
Msmooth_label step  tau  storm_factor bed_factor shore_factor             
grid          -1000 60.0 1.0          0.00       0.0           0.679939   
                                      0.01       0.0           0.652360   
                                      0.05       0.0           0.641652   
                                      0.10       0.0           0.619056   

                                                                      rho_p  \
Msmooth_label step  tau  storm_factor bed_factor shore_factor                 
grid          -1000 60.0 1.0          0.00       0.0           4.701109e-10   
                                      0.01       0.0           3.920428e-09   
                                      0.05       0.0           8.435670e-09   
                                      0.10       0.0           3.868281e-08   

                                                               norm_std  \
Msmooth_label step  tau  storm_factor bed_factor shore_factor             
grid          -1000 60.0 1.0          0.00       0.0           4.689978   
                                      0.01       0.0           4.533464   
                                      0.05       0.0           4.285618   
                                      0.10       0.0           4.059331   

                                                                norm_mag  \
Msmooth_label step  tau  storm_factor bed_factor shore_factor              
grid          -1000 60.0 1.0          0.00       0.0           10.357674   
                                      0.01       0.0            9.238952   
                                      0.05       0.0            7.887026   
                                      0.10       0.0            6.959880   

                                                               rel_std_obs  \
Msmooth_label step  tau  storm_factor bed_factor shore_factor                
grid          -1000 60.0 1.0          0.00       0.0              4.739295   
                                      0.01       0.0              4.739295   
                                      0.05       0.0              4.739295   
                                      0.10       0.0              4.739295   

                                                               rel_std_mod  \
Msmooth_label step  tau  storm_factor bed_factor shore_factor                
grid          -1000 60.0 1.0          0.00       0.0              2.145963   
                                      0.01       0.0              2.325526   
                                      0.05       0.0              2.575218   
                                      0.10       0.0              2.764181   

                                                                  r_log  \
Msmooth_label step  tau  storm_factor bed_factor shore_factor             
grid          -1000 60.0 1.0          0.00       0.0           0.643691   
                                      0.01       0.0           0.631243   
                                      0.05       0.0           0.611666   
                                      0.10       0.0           0.594804   

                                                                  r_sub  \
Msmooth_label step  tau  storm_factor bed_factor shore_factor             
grid          -1000 60.0 1.0          0.00       0.0           0.507480   
                                      0.01       0.0           0.509742   
                                      0.05       0.0           0.510127   
                                      0.10       0.0           0.509557   

                                                               norm_std_sub  \
Msmooth_label step  tau  storm_factor bed_factor shore_factor                 
grid          -1000 60.0 1.0          0.00       0.0              40.399557   
                                      0.01       0.0              39.084441   
                                      0.05       0.0              36.976899   

In [29]:
# Shore factor 0 to 0.1 gives norm_mag 10.4 down to 1.7
# And rho is slightly higher for 0.05.
# So that all makes sense.
all_metrics.loc[('grid',-1000,60,1.0,0.0,slice(None)),:]

rho  \
Msmooth_label step  tau  storm_factor bed_factor shore_factor             
grid          -1000 60.0 1.0          0.0        0.00          0.679939   
                                                 0.01          0.684441   
                                                 0.05          0.692308   
                                                 0.10          0.690822   

                                                                      rho_p  \
Msmooth_label step  tau  storm_factor bed_factor shore_factor                 
grid          -1000 60.0 1.0          0.0        0.00          4.701109e-10   
                                                 0.01          3.253395e-10   
                                                 0.05          1.682804e-10   
                                                 0.10          1.908990e-10   

                                                               norm_std  \
Msmooth_label step  tau  storm_factor bed_factor shore_factor             
grid          -1000 60.0 1.0          0.0        0.00          4.689978   
                                                 0.01          3.327140   
                                                 0.05          1.317900   
                                                 0.10          0.741532   

                                                                norm_mag  \
Msmooth_label step  tau  storm_factor bed_factor shore_factor              
grid          -1000 60.0 1.0          0.0        0.00          10.357674   
                                                 0.01           7.047513   
                                                 0.05           3.080196   
                                                 0.10           1.711122   

                                                               rel_std_obs  \
Msmooth_label step  tau  storm_factor bed_factor shore_factor                
grid          -1000 60.0 1.0          0.0        0.00             4.739295   
                                                 0.01             4.739295   
                                                 0.05             4.739295   
                                                 0.10             4.739295   

                                                               rel_std_mod  \
Msmooth_label step  tau  storm_factor bed_factor shore_factor                
grid          -1000 60.0 1.0          0.0        0.00             2.145963   
                                                 0.01             2.237427   
                                                 0.05             2.027766   
                                                 0.10             2.053823   

                                                                  r_log  \
Msmooth_label step  tau  storm_factor bed_factor shore_factor             
grid          -1000 60.0 1.0          0.0        0.00          0.643691   
                                                 0.01          0.651327   
                                                 0.05          0.652332   
                                                 0.10          0.645351   

                                                                  r_sub  \
Msmooth_label step  tau  storm_factor bed_factor shore_factor             
grid          -1000 60.0 1.0          0.0        0.00          0.507480   
                                                 0.01          0.494620   
                                                 0.05          0.418106   
                                                 0.10          0.300105   

                                                               norm_std_sub  \
Msmooth_label step  tau  storm_factor bed_factor shore_factor                 
grid          -1000 60.0 1.0          0.0        0.00             40.399557   
                                                 0.01             28.665330   
                                                 0.05             11.330193   

In [ ]:
# so an r_log of 0.717 is r2
all_metrics['r2_log']=all_metrics['r_log']**2
all_metrics.sort_values('r_log',ascending=False).head(10)

In [ ]:
# Best r_sub is also grid smoothed, step=-100000, but now
# tau=60.0. Again, not very sensitive to tau or storm_factor.
all_metrics.sort_values('r_sub',ascending=False).head(10)

In [ ]:
# Best magnitudes, including obs. outliers?
# This is all the place. Chooses some very noisy options,
# probably just happenstance to get close.  All pretty low rho
# and r_sub.
all_metrics['norm_mag_err']=np.abs(all_metrics['norm_mag']-1)
all_metrics.sort_values('norm_mag_err').head(10)

In [ ]:
# These mostly toss stormwater entirely.
# Again, poor rho and r_sub values.
all_metrics['norm_mag_sub_err']=np.abs(all_metrics['norm_mag_sub']-1)
all_metrics.sort_values('norm_mag_sub_err').head(10)

In [ ]:
fig=plt.figure(figsize=(6,2.5))
for smooth_type in ['grid','constant']:
    df_sub
    x=df_skill_smooth['smooth']

plt.plot(df_skill['tau_d'],df_skill['r'],label='Pearson $r$, full')
plt.plot(df_skill['tau_d'],df_skill['r_sub'],label='Pearson $r$, sub')

plt.plot(df_skill['tau_d'],df_skill['rho'],label=r'Spearman $\rho$, full')
plt.plot(df_skill['tau_d'],df_skill['rho_sub'],label=r'Spearman $\rho$, sub')

plt.plot(df_skill['tau_d'],df_skill['rlog'],label='Pearson $r$, full, log transform')
plt.plot(df_skill['tau_d'],df_skill['rlog_sub'],label='Pearson $r$, sub, log transform')
plt.legend(loc='upper left',bbox_to_anchor=[1.02,1.0],frameon=False)
plt.xlabel('Decay time scale (d)')
plt.axis(ymin=0,xmin=0)
fig.tight_layout()

Previously I got these results:
* Pearson $r$ is maximized by omitting any decay, as in $\tau\rightarrow\infty$
* Spearman $\rho$ is maximized by having a very short decay, 15h
* Log-transformed Pearson $r$ is maximized at around 25 days.

If I remove the top 2 and bottom 2 observed values, then Pearson
values come up to a level similar to log-pearson. Pearson
is still maximized by having little or no decay.

Re-running this, I don't see the local max in Pearson of the
log-transformed data.

What does it mean that Pearson correlation is highest at long decay
time scales, while Spearman is highest at short decay scales?

At short decay time scales, many of the predicted concentrations end up being
very small, substantially smaller than the observations even with no scaling
of the loads. This is penalized by Pearson, but for Spearman this correlates with
the lower concentration in the ocean compared to the estuary, and thus increases
skill.


In [ ]:
# What is Spearman just for getting two populations split correctly?
# Similar to what I get (for an offset that nearly makes the populations
# distinct)
# If the populations are different by a scale, then it takes a pretty 
# large scale (30?) to get a spearman of 0.7.

N=60
obs=np.random.random(N)
mod=np.random.random(N)

#off=0.8
#obs[:N//2]+=off
#mod[:N//2]+=off

fac=30
obs[:N//2]*=fac
mod[:N//2]*=fac

rho,p=spearmanr(obs,mod)
rho

In [ ]:
# Group samples - Central Bay, North Bay, South Bay, NMS.
def loc_to_region(s):
    if s.startswith('SPB') or s.startswith('SUB'): return 'NB'
    if s.startswith('SB') or s.startswith('LSB'): return 'SB'
    if s.startswith('GFNMS') or s.startswith('MBNMS') or s.startswith('CBNMS'): 
        return 'NMS'
    if s.startswith('CB') or s=='SFBay': return 'CB'
    raise Exception("No match for %s"%s)
    
def analyze_medians(df):
    dfm=df.loc[:,['region','obs','predicted']].melt(id_vars=['region']).set_index(['region','variable'])

    dfm.sort_index(inplace=True)

    def p_total(f):
        # Maximize the combined median test
        p_tot=1.0
        for region in ['CB','NB','SB','NMS']:
            obs=dfm.loc[ (region,'obs'), 'value']
            mod=f*dfm.loc[ (region,'predicted'), 'value']
            # is it better to choose just the modeled data where the observation
            # is valid?
            stat,p,M,cont=stats.median_test( obs[np.isfinite(obs)] ,
                                             mod[np.isfinite(mod)] )
            p_tot*=p
        return p_tot

    def cost(p):
        return -p_total( 10**p )

    from scipy.optimize import brute

    best=brute(cost,[[-4,1]],Ns=51)
    scale=10**best[0]
    p_tot=p_total(scale)
    print(f"P-total: {p_tot:.3f}  with scale: {1/scale:5f}")
    return p_tot,scale

In [ ]:
# Pulling in the old code from median comparisons.

manta_field='part_per_m2_nofiber'

for tau_d in [50]: # [0.5,1,1.5,2.,3,4,5,6,8,10,12,15,20,25,30,35,40,45,50,55,60]:
    print(f"Tau: {tau_d} days")
    tau=np.timedelta64(int(tau_d*86400),'s')
    df=predict_mantas(manta,tau,storm_factor=0.01)
    
    df=df[ df[manta_field].notnull() ].copy()
    
    regions=df['region']=df['SAMPLE LOCATION'].apply(loc_to_region)
    df=df.rename(columns={manta_field:'obs','C_model':'predicted'}).loc[:,['region','obs','predicted']]

    # Scale gets optimized to find the best nonparametric match
    p_tot,scale=analyze_medians(df)

    # This is not rigorous!
    obs_mag=1.0
    pred_mag=1/scale
    
    df['obs_frac']=df['obs'] / obs_mag
    df['pred_frac']=df['predicted'] / pred_mag

    dfm=df.loc[:,['region','obs_frac','pred_frac']].melt(id_vars=['region'])

    fig,ax=plt.subplots(1,1)# ,figsize=(10,3))
    sns.boxplot(x='region',y='value',hue='variable',data=dfm,fliersize=0,whis=0,
                ax=ax,bootstrap=1000)
    ax.axis(ymax=0.35,ymin=0)
    ax.legend(ax.artists,['Observed',f'Predicted/{pred_mag:.2f}'],loc='upper right')
    ax.set_ylabel('Particle m$^{-2}$')
    ax.set_xlabel('Region')
    
    ax.set_title(f'Tau={tau_d} days  $\\Pi \\rho$={p_tot:.2f}')
    df2=dfm.set_index(['region','variable'])
    p_tot=1.0
    for region in ['CB','NB','SB','NMS']:
        stat,p,M,cont=stats.median_test( df2.loc[ (region,'obs_frac'), 'value'] ,
                                         df2.loc[ (region,'pred_frac'), 'value'] )
        p_tot*=p
    print(f"{tau_d} days: ampl. error: {pred_mag/obs_mag:.3f} Median test net p: {p_tot:.5f}")
    
    print()
    fig.savefig(f'manta_comp_v05-tau{tau_d:.1f}.png',dpi=200)
    #break

In [ ]:
#plt.figure()
#plt.plot( df2.loc[('NB','obs_frac'),'value'], 'ro')
#plt.plot( df2.loc[('NB','pred_frac'),'value'],'go')

# Just 6 values...
display(df2.loc[('NB','obs_frac'),'value'])
display(df2.loc[('NB','pred_frac'),'value'])


In [ ]:
# For each manta sample, make a plot showing the conc. distribution at that time
# to debug what's up with the north bay samples:

tau_d=8
fac=39.81 # this is from analyze_medians.
six.moves.reload_module(conc_figure)

fig_dir=f"sample_wise_tau{tau_d}"
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)

tau=np.timedelta64(int(tau_d*86400),'s')
for idx,rec in manta.iterrows():
    print(idx)
    img_fn=os.path.join(fig_dir,f"sample-{idx}.png")
    fig,axs=plt.subplots(1,2,figsize=(11,7))
    fig.subplots_adjust(left=0,right=1,top=1,bottom=0,wspace=0.02)
    df=particles_for_date(rec.DATE).copy()

    age=df['time'] - df['rel_time']
    decay=np.exp( -age/tau ) 
    
    # No spatial weighting, but need the godin filter weights
    t_sample=np.datetime64(rec.DATE)
    # Noon, local, day of sampling
    t_center = t_sample+np.timedelta64(8,'h') + np.timedelta64(12,'h')
    delta_hours=((df['time']-t_center)/np.timedelta64(1,'h')).astype(np.int32)
    weight_time=godwin[delta_hours+godwin_offset_h]

    df['count']=weight_time * decay * df['mp_per_particle'] 

    weight=weight_time * decay * df['mp_per_particle'] 
        
    ds_conc=post.particles_to_conc(df,grid,'count')

    ds_smooth=ds_conc.copy()
    for _ in range(30):
        ds_smooth['conc']=('cell',), Msmooth.dot(ds_smooth.conc.values)
    
    cell=grid.select_cells_nearest([rec.x,rec.y])
        
    zoom=conc_figure.BayConcFigure.zoom
    
    if ( (rec.x>zoom[0]) and
        (rec.x<zoom[1]) and
        (rec.y>zoom[2]) and
        (rec.y<zoom[3])):
        FigCls=conc_figure.BayConcFigure
        FigCls.cax_loc=[0.82,0.25,0.03,0.35] # normalized to axis 
    else:
        FigCls=conc_figure.CoastalConcFigure
        
    for ds,ax in zip([ds_smooth,ds_conc],axs):
        cf=FigCls(ds,fig=fig,ax=ax,grid=grid)
        cf.ax.plot([rec.x],[rec.y],'k+')
        conc_pred=ds.conc.isel(cell=cell).item()
        conc_pred=conc_pred / fac

        cf.ax.text(rec.x,rec.y,
                   f"part_per_m2_nofiber\n obs={rec.part_per_m2_nofiber:.2e}\npred={conc_pred:.2e}")

        cf.ax.text(0.01,0.95,f"Manta sample: {rec.SampleID}",transform=cf.ax.transAxes,va='top')
    fig.savefig(img_fn,dpi=200)


In [ ]:
ds_conc

Next Steps
-- 

2. Think through how to present the comparisons. *ish*
3. What can we infer from these patterns? *ish*
4. Results appear worse, in terms of likelihood, than 
   for the old runs, probably driven by NB. Looks like 
   I am underpredicting NB.  Sample by sample, not terribly
   obvious what's up.  It's just 6 samples.
4. Would having different loss coefficients for WW and SW be worth
   attempting?
    - This is worth trying, and try maximizing the pearson with outliers
      tossed.
    - Does not make much difference.
5. Is it worth following a similar analysis to previously, lumping
   samples into embayments? *Blindly running that same code, it suggests
   that decay times >=20 days are best, with little difference beyond there.*
   I'm getting smaller net prob. values, though, and it seems to be driven
   by NB being underpredicted (relatively speaking) in the new runs. Any chance
   I'm missing something in the north? The one map plot above looks okay, but compare
   for wet weather.
   
What about applying a much more aggressive smoothing, going implicit?

In [ ]:
six.moves.reload_module(conc_figure)

conc_figure.BayConcFigure(conc_ds,grid=grid,cmap=cmap)

Closer look at distributions
---

Tau constant at 25:

The predictions with the best spearman were grid smoothing -10000.
But the log-normal shape here is pretty terrible.

A moderately better log-normal comes up with no smoothing.

The best match in distribution is with no smoothing, tau=10,
storm_factor 0.05 or 0.01.

Given the amount of smoothing already taking place in time,
and how much the results here are affected by smoothing in
space, it may not be a great comparison.

In [ ]:
import seaborn as sns
import scipy.stats

fig,axs=plt.subplots(2,1)

mp=manta_pred_master
sel_tau=mp.tau==10
sel_smooth=mp.Msmooth_label=='constant'
sel_step=mp.step==0
sel_storm=mp.storm_factor==0.01
sel=sel_tau & sel_smooth & sel_step & sel_storm
df=manta_pred_master[sel]

nz=df['part_per_m2_nofiber']>0
kde=False
ret=sns.histplot(df['part_per_m2_nofiber'][nz],
                 log_scale=True,kde=kde,bins=25,ax=axs[0])
shape,loc,scale=stats.lognorm.fit(df['part_per_m2_nofiber'][nz],floc=0)
x=10**np.linspace(-5,2,300)
ln_pred=stats.lognorm.pdf(x,shape,loc=loc,scale=scale)
axs[0].plot(x,x*ln_pred*nz.sum()/2,color='tab:orange',label='Log-normal')

# ---- same, for model output
pred=df['C_model'].values
pred=pred[pred>0]

ret=sns.histplot(pred,log_scale=True,kde=kde,bins=25,ax=axs[1])
m_shape,m_loc,m_scale=stats.lognorm.fit(pred,floc=0)
ln_pred=stats.lognorm.pdf(x,m_shape,loc=m_loc,scale=m_scale)
axs[1].plot(x,x*ln_pred*nz.sum()/2,color='tab:orange',label='Log-normal')

print("Obs scale: ",scale)
print("Mod scale: ",m_scale)
print("Obs shape: ",shape)
print("Mod shape: ",m_shape)

In [ ]:
mp=manta_pred_master
sel_tau=mp.tau==60
sel_smooth=mp.Msmooth_label=='grid'
sel_step=mp.step==-100000
sel_storm=mp.storm_factor==1.0
sel=sel_tau & sel_smooth & sel_step & sel_storm
df=manta_pred_master[sel]

import statsmodels.formula.api as smf
from scipy.stats import spearmanr



In [ ]:
# statsmodels on log-transform
n_low=n_high=0
obs_rank=np.argsort(np.argsort(df['part_per_m2_nofiber']))
obs_sel=(obs_rank>=n_low) & (obs_rank<len(df)-n_high)
mod_rank=np.argsort(np.argsort(df['C_model']))
mod_sel=(mod_rank>=n_low) & (mod_rank<len(df)-n_high)

sel_nz=(df['part_per_m2_nofiber']>0)&(df['C_model']>0).values
sel=obs_sel&mod_sel&sel_nz
print(f"N={len(rank)}  Nsel={sel.sum()}")

data=df[sel].copy()
data['log_mod']=np.log(data['C_model'])
data['log_obs']=np.log(data['part_per_m2_nofiber'])

res=smf.ols(formula='log_mod ~ log_obs', data=data).fit()
res.summary()

In [ ]:
rho,rho_p=spearmanr(data['C_model'],data['part_per_m2_nofiber'])
print(f"Spearman:  rho={rho}  p-value={rho_p}")